In [1]:
# Author: Eric Chandler <echandler@uchicago.edu>
# Description: Trying to write some computer code to make sense of the Durlauf-Seshadri paper

## Goals
Implement the Durlauf-Seshadri model as a simulation, reproduce the expected stylized facts, identify causal mechanism, test implications of extending model or relaxing certain assumptions.
## Argument
The paper's main arguments are 1) segregation causes and aggravates inequality 2) and that inequality decreases subsequent generations' mobility.
## Stylized Facts
* intertemporal great gadby curve emerges
## Key Assumptions
* no marriage or inter-family income sharing
* one child, no elders, no population growth or shrink
* income is certain and fully attained at time of child investment (no credit constraint)
* no direct parental investment (only via neighborhood)
* parental utility depends on child expected income, not on child expected utility or human capital
## Interesting Questions
* credit constraints are an alternative identifiable mechanism

## Model
cross-sectional inequality -> neighborhood differences

segregation:= increasing function of cross-sectional inequality

mobility := parental input + neighborhood quality

agents in firm/school split earninge evenly. no additional benefits for most productive person.

### Optional
* assortive matching of workers by skill because its more efficient overall
* assortive matching of students to schools because its more efficient overall
* spatial segregation




In [3]:
import numpy as np
import pandas as pd
from numpy.random import default_rng
import scipy

## Define Model Processes

In [ ]:
# Parameters
N_FAMILIES = 100
N_TIMESTEPS = 100
INCOME_GROWTH = 0   # alpha in eq(1)
INCOME_MOBILITY_COEF = 0  # beta in eq(1)
INCOME_NOISE_AUTOREG = 1 
rng = default_rng(seed=12345)

In [ ]:
def generate_white_noise(rng, size, mean=0, var=1):
    """returns `size` random samples"""
    # any white noise process will do
    return rng.standard_normal(mean, var, size)

def generate_income_noise(rng, parent_noise=None):
    """aka epsilon, a MA(1) process.  from eq (1)"""
    noise = generate_white_noise(rng, N_FAMILIES)
    if parent_noise is not None:
        noise += INCOME_NOISE_AUTOREG * parent_noise
    return noise
    
def generate_mobility(parent_income):
    """ beta in eq(1) and eq(2) """
    # follows eq(1) for now
    return np.full(parent_income.shape, INCOME_MOBILITY_COEF)

def transmit_income(rng, parent_income, parent_noise):
    """one intergenerational timestep. eq(1) """
    mobility = generate_mobility(parent_income)
    noise = generate_income_noise(rng, parent_noise)
    offspring_income = INCOME_GROWTH + mobility * parent_income + noise
    return offspring_income, noise

def initialize_income(rng):
    """ create initial income distribution """
    # this is arbitrary right now
    income = np.full(N_FAMILIES, 1)
    return income, np.zeros(income.shape)

def run():
    income = np.zeros(N_TIMESTEPS, N_FAMILIES)
    parent_income, parent_noise = initialize_income(rng)
    income[0, :] = parent_income

    for t in range(1, N_TIMESTEPS):
        child_income, child_noise = transmit_income(rng, income[t-1, :], parent_noise)
        income[t, :] = child_income
        parent_noise = child_noise
    
    pd.DataFrame(income).plot()
                                            

## Previous code describes paper up through page 5

In [ ]:
# Parameters
N_FAMILIES = 100
N_TIMESTEPS = 100
income_noise_theta = 1
cobbdouglass = np.array(.5, .5)
# Model Objects
income = np.array(N_TIMESTEPS, N_FAMILIES)
income_noise = np.array(N_TIMESTEPS, N_FAMILIES)
income_shocks = np.array(N_TIMESTEPS, N_FAMILIES)
skills = np.array(N_TIMESTEPS, N_FAMILIES)
utility = np.array(N_TIMESTEPS, N_FAMILIES)
info = np.array(N_TIMESTEPS)
# Initialize statistical objects
income[0] = initial_income()
# Step
def expected_income(income, info, t):
    return income[t+1] | info[t]
for t in N_TIMESTEPS:
    income_eps = (np.mean(income[:t-1]) + income_noise[t] + income_noise_theta * income_noise[t-1])
    income[t] = alpha + beta * (skills[t-1] * income_shocks[t]) + income_eps
    utility[t] = cobbdouglas * np.array(np.log(consumption), np.log(expected_income(income, info, t)))
    consumption = np.array(N_FAMILIES)
    taxes = np.array(N_FAMILIES)